In [0]:
%sql
TRUNCATE TABLE olist_db.gold.daily_sales;
INSERT INTO olist_db.gold.daily_sales
SELECT
  DATE(o.order_purchase_timestamp) AS order_date,
  SUM(oi.price + oi.freight_value) AS total_revenue,
  COUNT(DISTINCT o.order_id) AS total_orders,
  current_timestamp() AS ETL_Date
FROM olist_db.silver.orders o
JOIN olist_db.silver.order_items oi
ON o.order_id = oi.order_id
GROUP BY order_date
ORDER BY order_date DESC;

In [0]:
%sql
TRUNCATE TABLE olist_db.gold.seller_performance;
INSERT INTO olist_db.gold.seller_performance
SELECT
  s.seller_id,
  COUNT(DISTINCT o.order_id) AS total_orders,
  SUM(oi.price) AS revenue,
  AVG(DATEDIFF(
      o.order_delivered_customer_date,
      o.order_purchase_timestamp
  )) AS avg_delivery_days,
  current_timestamp() AS ETL_Date
FROM olist_db.silver.orders o
JOIN olist_db.silver.order_items oi ON o.order_id = oi.order_id
JOIN olist_db.silver.sellers s ON oi.seller_id = s.seller_id
GROUP BY s.seller_id;


In [0]:
%sql
TRUNCATE TABLE olist_db.gold.delivery_vs_reviews;
INSERT INTO olist_db.gold.delivery_vs_reviews
SELECT
  r.review_score,
  AVG(DATEDIFF(
      o.order_delivered_customer_date,
      o.order_purchase_timestamp
  )) AS avg_delivery_days,
  current_timestamp() AS ETL_Date
FROM olist_db.silver.order_reviews r
JOIN olist_db.silver.orders o
ON r.order_id = o.order_id
GROUP BY r.review_score;


In [0]:
%sql
TRUNCATE TABLE olist_db.gold.state_wise_revenue;
INSERT INTO olist_db.gold.state_wise_revenue
SELECT
  c.customer_state,
  SUM(oi.price + oi.freight_value) AS revenue,
  current_timestamp() AS ETL_Date
FROM olist_db.silver.customers c
JOIN olist_db.silver.orders o ON c.customer_id = o.customer_id
JOIN olist_db.silver.order_items oi ON o.order_id = oi.order_id
GROUP BY c.customer_state;


In [0]:
%sql
TRUNCATE TABLE olist_db.gold.monthly_sales;
INSERT INTO olist_db.gold.monthly_sales
SELECT
  date_trunc('month', o.order_purchase_timestamp) AS order_month,
  SUM(oi.price + oi.freight_value) AS total_revenue,
  COUNT(DISTINCT o.order_id) AS total_orders,
  current_timestamp() AS ETL_Date
FROM olist_db.silver.orders o
JOIN olist_db.silver.order_items oi
ON o.order_id = oi.order_id
GROUP BY order_month
ORDER BY order_month;


In [0]:
%sql
TRUNCATE TABLE olist_db.gold.top_products;
INSERT INTO olist_db.gold.top_products
SELECT
  p.product_id,
  pc.product_category_name_english,
  SUM(oi.price) AS revenue,
  COUNT(*) AS units_sold,
  current_timestamp() AS ETL_Date
FROM olist_db.silver.order_items oi
JOIN olist_db.silver.products p
ON oi.product_id = p.product_id
JOIN olist_db.silver.product_category_name_translation pc
ON pc.product_category_name = p.product_category_name
GROUP BY p.product_id, pc.product_category_name_english
ORDER BY revenue DESC;


In [0]:
%sql
TRUNCATE TABLE olist_db.gold.customer_ltv;
INSERT INTO olist_db.gold.customer_ltv
SELECT
  c.customer_id,
  SUM(oi.price + oi.freight_value) AS lifetime_value,
  COUNT(DISTINCT o.order_id) AS total_orders,
  current_timestamp() AS ETL_Date
FROM olist_db.silver.customers c
JOIN olist_db.silver.orders o ON c.customer_id = o.customer_id
JOIN olist_db.silver.order_items oi ON o.order_id = oi.order_id
GROUP BY c.customer_id;


In [0]:
%sql
TRUNCATE TABLE olist_db.gold.payment_method_stats;
INSERT INTO olist_db.gold.payment_method_stats
SELECT
  payment_type,
  COUNT(DISTINCT order_id) AS total_orders,
  SUM(payment_value) AS total_paid,
  current_timestamp() AS ETL_Date
FROM olist_db.silver.order_payments
GROUP BY payment_type;


In [0]:
%sql
TRUNCATE TABLE olist_db.gold.delivery_performance;
INSERT INTO olist_db.gold.delivery_performance
SELECT
  CASE
    WHEN DATEDIFF(order_delivered_customer_date, order_purchase_timestamp) <= 3 THEN 'Fast'
    WHEN DATEDIFF(order_delivered_customer_date, order_purchase_timestamp) <= 7 THEN 'On Time'
    ELSE 'Delayed'
  END AS delivery_category,
  COUNT(*) AS total_orders,
  current_timestamp() AS ETL_Date
FROM olist_db.silver.orders
WHERE order_delivered_customer_date IS NOT NULL
GROUP BY delivery_category;


In [0]:
%sql
TRUNCATE TABLE olist_db.gold.review_distribution;
INSERT INTO olist_db.gold.review_distribution
SELECT
  review_score,
  COUNT(*) AS review_count,
  current_timestamp() AS ETL_Date
FROM olist_db.silver.order_reviews
GROUP BY review_score
ORDER BY review_score;


In [0]:
%sql
TRUNCATE TABLE olist_db.gold.seller_state_revenue;
INSERT INTO olist_db.gold.seller_state_revenue
SELECT
  s.seller_state,
  SUM(oi.price) AS revenue,
  current_timestamp() AS ETL_Date
FROM olist_db.silver.sellers s
JOIN olist_db.silver.order_items oi
ON s.seller_id = oi.seller_id
GROUP BY s.seller_state;


In [0]:
%sql
TRUNCATE TABLE olist_db.gold.category_performance;
INSERT INTO olist_db.gold.category_performance
SELECT
  pc.product_category_name_english,
  SUM(oi.price) AS revenue,
  COUNT(*) AS total_items_sold,
  current_timestamp() AS ETL_Date
FROM olist_db.silver.products p
JOIN olist_db.silver.order_items oi
ON p.product_id = oi.product_id
JOIN olist_db.silver.product_category_name_translation pc
ON pc.product_category_name = p.product_category_name
GROUP BY pc.product_category_name_english;
